In [2]:
from ydata_profiling import ProfileReport
import pandas as pd
%pip install pdfkit
import pdfkit
import pickle



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os, subprocess
pkl_path = '/workspaces/Project-Team-01-Samsung-Innovation-Campus-2025/data/df_ouvidoria_aneel_limpo.pkl'
def is_lfs_pointer(path):
    try:
        with open(path, 'rb') as f:
            start = f.read(16)
        return start.startswith(b'version ')
    except Exception:
        return False
# se for ponteiro, faz git lfs pull com path relativo ao repositório
if is_lfs_pointer(pkl_path):
    rel = os.path.relpath(pkl_path, os.getcwd())
    print(f'Detected Git LFS pointer for {rel}, running git lfs pull...')
    try:
        subprocess.run(['git','lfs','pull','--include=' + rel], check=True)
        print('git lfs pull completed')
    except subprocess.CalledProcessError as e:
        print('git lfs pull failed:', e)
else:
    print('File looks like binary pickle (not LFS pointer)')


In [3]:
df_ouvidoria_aneel = pd.read_pickle('/workspaces/Project-Team-01-Samsung-Innovation-Campus-2025/data/df_ouvidoria_aneel_limpo.pkl')
df_ouvidoria_aneel.head()

,DatGeracaoConjuntoDados,SigUF,NomMunicipio,NomCategoria,NomSubCategoria,NomTipologia,NomDecisao,DscSituacao,DtCriacao,NumQtdReclamacoesDia
0,2025-09-25,RO,ALTA FLORESTA D'OESTE,Informação,Qualidade do Fornecimento,Oscilação de Tensão,Sem decisão,Finalizada por Script,2025-01-21,1.0
1,2025-09-25,RO,ALTA FLORESTA D'OESTE,Reclamações,Cobranças,Devol Valores,Improcedente,Encerrada,2025-02-24,4.0
2,2025-09-25,RO,ALTA FLORESTA D'OESTE,Reclamações,Qualidade do Fornecimento,Oscilação de Tensão,Sem decisão,Novos Questionamentos à Concessionária,2025-01-27,1.0
3,2025-09-25,RO,ALTA FLORESTA D'OESTE,Informação,Faturamento,Variação de Consumo,Sem decisão,Finalizada por Script,2025-09-09,1.0
4,2025-09-25,RO,ALTA FLORESTA D'OESTE,Informação,Serviços Técnicos,Devol Antecipação Obras,Sem decisão,Encerrada,2025-08-18,1.0


In [23]:
df_ouvidoria_aneel.drop(columns=['NomMunicipio'], inplace=True)

In [24]:
categories  = lambda df_ouvidoria_aneel: df_ouvidoria_aneel.columns[df_ouvidoria_aneel.dtypes == 'category']

In [33]:
from scipy.stats import chi2_contingency

# Create contingency table between NomDecisao and DscSituacao
ct = pd.crosstab(df_ouvidoria_aneel['NomDecisao'], df_ouvidoria_aneel['NomSubCategoria'])

chi2, p, dof, expected = chi2_contingency(ct)
print(f'Chi2={chi2:.2f}, p-valor={p:.4f}')
if p < 0.05:
    # Create a dataframe to store results
    results = []

    # Get categorical columns
    categorical_cols = df_ouvidoria_aneel.select_dtypes(include='category').columns

    # Calculate contingency tables and chi-square tests for all pairs
    for i, col1 in enumerate(categorical_cols):
        for j, col2 in enumerate(categorical_cols):
            if i < j:  # Avoid duplicates and self-comparisons
                ct = pd.crosstab(df_ouvidoria_aneel[col1], df_ouvidoria_aneel[col2])
                chi2, p, dof, expected = chi2_contingency(ct)
                
                # Calculate Cramér's V
                n = ct.values.sum()
                phi2 = chi2/n
                r, k = ct.shape
                cramers_v = np.sqrt(phi2 / min(k-1, r-1))
                
                results.append({
                    'Variable1': col1,
                    'Variable2': col2,
                    'Chi2': chi2,
                    'p_value': p,
                    'Cramers_V': cramers_v,
                    'Significant': p < 0.05
                })

    # Create dataframe with results
    results_df = pd.DataFrame(results)
    print(results_df)
else:
    print('Sem associação significativa')

Chi2=426586.77, p-valor=0.0000
          Variable1        Variable2          Chi2  p_value  Cramers_V  \
0             SigUF     NomCategoria  2.478852e+05      0.0   0.113627   
1             SigUF  NomSubCategoria  1.851945e+05      0.0   0.065475   
2             SigUF     NomTipologia  6.131614e+05      0.0   0.070095   
3             SigUF       NomDecisao  9.547355e+04      0.0   0.099727   
4             SigUF      DscSituacao  4.473928e+05      0.0   0.078829   
5      NomCategoria  NomSubCategoria  1.474278e+07      0.0   0.876284   
6      NomCategoria     NomTipologia  1.550873e+07      0.0   0.898759   
7      NomCategoria       NomDecisao  1.537798e+06      0.0   0.400239   
8      NomCategoria      DscSituacao  4.263071e+06      0.0   0.471212   
9   NomSubCategoria     NomTipologia  4.062300e+07      0.0   0.969728   
10  NomSubCategoria       NomDecisao  4.265868e+05      0.0   0.210802   
11  NomSubCategoria      DscSituacao  5.525964e+05      0.0   0.113101   
12     

In [34]:
import numpy as np
n = ct.values.sum()
phi2 = chi2/n

# Use the existing results dataframe
print("Chi-square test results for all categorical variable pairs:")
print(results_df)

# You can also filter for specific criteria if needed
print("\nVariables with strongest associations (Cramér's V > 0.5):")
strong_associations = results_df[results_df['Cramers_V'] > 0.5]
print(strong_associations)
cramers_v = np.sqrt(phi2 / min(k-1, r-1))
print(f"Cramér’s V = {cramers_v:.3f}")

Chi-square test results for all categorical variable pairs:
          Variable1        Variable2          Chi2  p_value  Cramers_V  \
0             SigUF     NomCategoria  2.478852e+05      0.0   0.113627   
1             SigUF  NomSubCategoria  1.851945e+05      0.0   0.065475   
2             SigUF     NomTipologia  6.131614e+05      0.0   0.070095   
3             SigUF       NomDecisao  9.547355e+04      0.0   0.099727   
4             SigUF      DscSituacao  4.473928e+05      0.0   0.078829   
5      NomCategoria  NomSubCategoria  1.474278e+07      0.0   0.876284   
6      NomCategoria     NomTipologia  1.550873e+07      0.0   0.898759   
7      NomCategoria       NomDecisao  1.537798e+06      0.0   0.400239   
8      NomCategoria      DscSituacao  4.263071e+06      0.0   0.471212   
9   NomSubCategoria     NomTipologia  4.062300e+07      0.0   0.969728   
10  NomSubCategoria       NomDecisao  4.265868e+05      0.0   0.210802   
11  NomSubCategoria      DscSituacao  5.525964e+05  

In [37]:
%pip install prince
%pip install matplotlib
from prince import MCA
import matplotlib.pyplot as plt

# Use the existing categorical columns from the dataframe
# Select a subset of categorical columns for MCA (to avoid computational issues)
selected_cols = ['NomCategoria', 'NomSubCategoria', 'NomDecisao']

mca = MCA(n_components=2, random_state=42)
mca_fit = mca.fit(df_ouvidoria_aneel[selected_cols])

# coordenadas das categorias
coords = mca_fit.column_coordinates(df_ouvidoria_aneel[selected_cols])
print(coords.head())

# plotar resultados
ax = mca_fit.plot_coordinates(X=df_ouvidoria_aneel[selected_cols],
                              ax=None,
                              figsize=(10,8),
                              show_row_points=False,
                              show_column_points=True,
                              column_points_size=50)
ax.set_title('MCA das Variáveis Categóricas')
plt.show()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.2/731.2 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [prince]

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
                                        0          1
NomCategoria__Denúncias         24.708281   3.039906
NomCategoria__Elogio            -5.817188  49.366516
NomCategoria__Informação        -0.094736  -0.022759
NomCategoria__Reclamações        0.112606  -0.033342
NomCategoria__Sugestão/Crítica  -5.717454  46.280038


AttributeError: 'MCA' object has no attribute 'plot_coordinates'